# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [1]:
import sqlite3

In [2]:
con = sqlite3.connect("../data/Chinook_Sqlite.sqlite")
cur = con.cursor()

cur.execute('''
SELECT firstname, lastname 
FROM Customer 
WHERE country = ?
''', ['Canada'])

# .fetchone() - первый или единственный
# .fetchmany(n) - первые несколько
# .fetchall() - всеЫ
result = cur.fetchall()
print(result)

[('François', 'Tremblay'), ('Mark', 'Philips'), ('Jennifer', 'Peterson'), ('Robert', 'Brown'), ('Edward', 'Francis'), ('Martha', 'Silk'), ('Aaron', 'Mitchell'), ('Ellie', 'Sullivan')]


2. Найти и вывести на экран названия всех альбомов группы Accept

In [3]:
cur = con.cursor()

cur.execute('''
SELECT Title 
FROM Artist  t1 
INNER JOIN Album t2 
ON t1.ArtistId = t2.ArtistId 
WHERE t1.Name = ?
''', ["Accept"])

# .fetchall()
for item in cur.fetchall():
    print(item[0])

Balls to the Wall
Restless and Wild


3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [4]:
con = sqlite3.connect('test.db')
cur = con.cursor()

cur.execute('''
CREATE TABLE Student(id INTEGER, name VARCHAR(50))
''')

cur.execute('''
INSERT INTO Student(id , name) VALUES (1, 'Nikita'),('2','Robot')
''')

con.commit()

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [2]:
con = sqlite3.connect('recipes.db')
cur = con.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [3]:
cur.execute('''
CREATE TABLE Recipe(
    id INTEGER,
    name TEXT,
    minutes REAL,
    submitted INTEGER,
    description TEXT,
    n_ingredients INTEGER,
    PRIMARY KEY(id AUTOINCREMENT)
);
''')

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [4]:
cur.execute('''
CREATE TABLE Review(
    id INTEGER,
    user_id INTEGER,
    recipe_id INTEGER NOT NULL,
    date TEXT,
    rating INTEGER,
    review TEXT,
    PRIMARY KEY(id AUTOINCREMENT),
    FOREIGN KEY(recipe_id) REFERENCES Recipe(id)
);
''')

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [5]:
import pandas as pd

In [6]:
reviews = pd.read_csv('../data/reviews_sample.csv', sep=',')
recipes = pd.read_csv('../Lab5/recipes_sample_with_tags_ingredients.csv', sep=',')

In [7]:
for i, row in recipes.iterrows():
    rows = row.loc[['name', 'minutes', 'submitted', 'description', 'n_ingredients']]
    cur.execute(
        '''
        INSERT INTO
        Recipe (name, minutes, submitted, description, n_ingredients)
        VALUES(?, ?, ?, ?, ?)
        ''',
    rows)

con.commit()

In [8]:
for i, row in reviews.iterrows():
    rows = row.loc[['user_id', 'recipe_id', 'date', 'rating', 'review']]
    cur.execute(
        '''
        INSERT INTO
        Review (user_id, recipe_id, date, rating, review)
        VALUES (?, ?, ?, ?, ?)
        ''',
    rows)

con.commit()

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [12]:
cur = cur.execute(
    '''
    SELECT *
    FROM Recipe
    WHERE n_ingredients = ?
    ''', [10])

cur.fetchmany(5)

[(14,
  'blepandekager   danish   apple pancakes',
  50.0,
  '2013-07-08',
  "this recipe has been posted here for play in zwt9 - scandinavia.  this recipe was found at website: mindspring.com - christian's danish recipes.",
  10),
 (22,
  'kelly s creamy cheddar pea salad',
  20.0,
  '2005-06-09',
  "i'm not a big fan of peas, but like them in things. this salad is one of my favorites. happy to share!",
  10),
 (25,
  'middle eastern   twice baked potatoes',
  110.0,
  '2005-05-02',
  'a delicious yet low-fat twice-baked potato side dish with a blend of classic middle eastern flavours: chickpeas, cumin and coriander.  serve with roasts or with your favourite middle eastern main course dish.  adapted from "practical cookery: low fat".  these potatoes could also be served on their own as a light vegetarian meal. in view of the comments by reviewers about the potatoes being a bit dry, i have added a tablespoon of tahini. i have also added some garlic. i\'m not sure how the recipe escaped

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [13]:
cur = cur.execute(
    '''
    SELECT name
    FROM Recipe
    WHERE minutes = (
        SELECT MAX(minutes)
        FROM Recipe
    )
    ''')

cur.fetchone()

('blueberry liqueur',)

7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [18]:
pecipe_id = input('Введите id рецепта ')

cur = cur.execute(
    '''
    SELECT *
    FROM Recipe
    WHERE id = ?
    ''', [pecipe_id])

result = cur.fetchone()

if result:
    print(result)
else:
    print('Такого рецепта нет')

Такого рецепта нет


8. Найдите кол-во отзывов с рейтингом 5.

In [22]:
cur = cur.execute(
    '''
    SELECT COUNT(id)
    FROM Review
    WHERE rating = ?
    ''', [5])

cur.fetchone()[0]

91361

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [28]:
cur = cur.execute('''
    SELECT COUNT(*)
    FROM (
        SELECT *
        FROM recipe rc
        LEFT JOIN review rv
        ON rc.id = rv.recipe_id
        GROUP BY rc.id
        HAVING 0 = COUNT(rv.rating < ?)
    )
    ''', [4])

cur.fetchone()[0]

28031

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [31]:
cur = cur.execute('''
    SELECT COUNT(*)
    FROM recipe
    WHERE submitted LIKE ? AND minutes >= ?
    ''', ['2010%', 15])

cur.fetchone()[0]

1319

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [34]:
cur = cur.execute('''
    SELECT rc.id, rc.name, rv.user_id, rv.date, rv.rating
    FROM recipe rc
    LEFT JOIN review rv
    ON rc.id = rv.recipe_id
    WHERE rc.n_ingredients >= ?
    ORDER BY rc.id
    ''', [3])

cur.fetchall()

[(1, 'george s at the cove  black bean soup', None, None, None),
 (2, 'healthy for them  yogurt popsicles', None, None, None),
 (3, 'i can t believe it s spinach', None, None, None),
 (4, 'italian  gut busters', None, None, None),
 (5, 'love is in the air  beef fondue   sauces', None, None, None),
 (6, 'mennonite  corn fritters', None, None, None),
 (7, 'open sesame  noodles', None, None, None),
 (8, 'say what   banana sandwich', None, None, None),
 (9, '1 in canada chocolate chip cookies', None, None, None),
 (10, '412 broccoli casserole', None, None, None),
 (11, '25 000 casserole', None, None, None),
 (12, '250 00 chocolate chip cookies', None, None, None),
 (13, 'ara s potato  oup', None, None, None),
 (14, 'blepandekager   danish   apple pancakes', None, None, None),
 (15, 'add in anything  muffins', None, None, None),
 (16, 'better than tofu  cheesecake', None, None, None),
 (17, 'burek  or feta cheese  phyllo pie', None, None, None),
 (18, 'da best  chicago style italian beef', 